<h2>Webscraping</h2>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

response = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/')
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')
links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if href.endswith('.json') and not href =='-1.json' and not href == '1.json':
        links.append(f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/{href}")

def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict,list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)

        return arr
    
    values = extract(obj, arr, key)
    return values

all_champions_Names = []
all_champions_Info = []

for url in links:
    champion_json = requests.get(url).json()
    
    name = json_extract(champion_json, 'alias')
    champion_name = { 'name' : name[0]}
    champion_info = json_extract(champion_json, 'tacticalInfo')
    champion_playstyle = json_extract(champion_json, 'playstyleInfo')

    champion_playstyle = champion_playstyle.pop()
    champion_info = champion_info.pop()
    champion_name.update(champion_info)
    champion_name.update(champion_playstyle)

    all_champions_Info.append(champion_name)

all_champions_df = pd.DataFrame(all_champions_Info)

all_champions_df.drop(['style'], axis=1, inplace=True)

<h4>scraping u.gg champs wr, pickrate, banrate, tier</h4>

In [167]:
def standardizeNames(name):
    patternApos = r"([^']+)'([^']+)"
    name = re.sub(patternApos, r"\1\2", name)

    patternSpace = r'\s+'
    name = re.sub(patternSpace, '', name)

    patternPeriod = r'\.(\w)'
    name = re.sub(patternPeriod, r'\1', name)

    return name

In [168]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get('https://u.gg/lol/champions', headers=headers)
response.raise_for_status()  # Check if the request was successful
soup = BeautifulSoup(response.text, 'html.parser')
baseURL = "https://u.gg/"

champContainer = soup.find('div', class_='champion-home-page')
champion_data = []
if champContainer:
    links = champContainer.find_all('a')

    for link in links:
        url = baseURL + link['href']
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200:
            html = resp.text
            soup1 = BeautifulSoup(html, 'html.parser')
            rankingStats = soup1.find('div', class_='champion-ranking-stats-normal')
            if rankingStats:
                text = rankingStats.text.strip()
                #scrapedData.append(rankingStats.text.strip())
                tier_pattern = r"([A-S]\+?Tier)"
                redTier_pattern = r"([A-S]\+)"
                win_rate_pattern = r"(\d+(\.\d+)?%)Win Rate"
                #r"((\d+(\.\d+)?|\d)%)Win Rate"
                rank_pattern = r"(\d+ / \d+)Rank"
                pick_rate_pattern = r"(\d+\.\d+%)Pick Rate"
                ban_rate_pattern = r"(\d+\.\d+%)Ban Rate"
                matches_pattern = r"(\d+,?\d*,?\d*)Matches"
                
                # Apply patterns
                tier = re.search(tier_pattern, text).group(1)
                win_rate = re.search(win_rate_pattern, text).group(1)
                rank = re.search(rank_pattern, text).group(1)
                pick_rate = re.search(pick_rate_pattern, text).group(1)
                ban_rate = re.search(ban_rate_pattern, text).group(1)
                matches = re.search(matches_pattern, text).group(1)

                numbers = rank.split('/')

                # Strip whitespace and convert to integers
                numerator = int(numbers[0].strip())
                denominator = int(numbers[1].strip())

                # Perform division
                rank = f"{((numerator / denominator)*100):.2f}%"

                name = soup1.find('span', class_='champion-name').text.strip()
                name = standardizeNames(name)

                champion_data.append({
                'name' : name,
                'tier': tier,
                'win_rate': win_rate,
                'rank': rank,
                'pick_rate': pick_rate,
                'ban_rate': ban_rate,
                'matches': matches
                })



In [169]:
champion_data

[{'name': 'Aatrox',
  'tier': 'ATier',
  'win_rate': '49.96%',
  'rank': '64.41%',
  'pick_rate': '11.4%',
  'ban_rate': '16.2%',
  'matches': '286,783'},
 {'name': 'Ahri',
  'tier': 'CTier',
  'win_rate': '49.66%',
  'rank': '67.24%',
  'pick_rate': '8.9%',
  'ban_rate': '10.1%',
  'matches': '224,349'},
 {'name': 'Akali',
  'tier': 'DTier',
  'win_rate': '49%',
  'rank': '82.76%',
  'pick_rate': '4.8%',
  'ban_rate': '12.0%',
  'matches': '119,859'},
 {'name': 'Akshan',
  'tier': 'S+Tier',
  'win_rate': '52.48%',
  'rank': '5.17%',
  'pick_rate': '2.7%',
  'ban_rate': '8.6%',
  'matches': '67,908'},
 {'name': 'Alistar',
  'tier': 'STier',
  'win_rate': '50.75%',
  'rank': '27.27%',
  'pick_rate': '6.3%',
  'ban_rate': '2.3%',
  'matches': '158,238'},
 {'name': 'Amumu',
  'tier': 'ATier',
  'win_rate': '50.84%',
  'rank': '27.45%',
  'pick_rate': '3.4%',
  'ban_rate': '1.0%',
  'matches': '85,535'},
 {'name': 'Anivia',
  'tier': 'STier',
  'win_rate': '52.23%',
  'rank': '13.79%',
  '

In [170]:
champion_data_df = pd.DataFrame(champion_data)
champion_data_df

,name,tier,win_rate,rank,pick_rate,ban_rate,matches
0,Aatrox,ATier,49.96%,64.41%,11.4%,16.2%,"286,783"
1,Ahri,CTier,49.66%,67.24%,8.9%,10.1%,"224,349"
2,Akali,DTier,49%,82.76%,4.8%,12.0%,"119,859"
3,Akshan,S+Tier,52.48%,5.17%,2.7%,8.6%,"67,908"
4,Alistar,STier,50.75%,27.27%,6.3%,2.3%,"158,238"
...,...,...,...,...,...,...,...
162,Zeri,CTier,49.72%,59.26%,12.0%,4.6%,"300,497"
163,Ziggs,DTier,47.88%,93.10%,1.4%,0.3%,"35,191"
164,Zilean,ATier,50.25%,40.91%,2.3%,0.7%,"57,974"
165,Zoe,ATier,50.89%,32.76%,2.5%,1.2%,"61,492"


<h2>Data Base</h2>

In [9]:
from pangres import upsert
from sqlalchemy import text, create_engine

db_username = "postgres"
db_password = "123123123"
db_host = "localhost"
db_port = "5432"
db_name = "lol_analytics"

def create_db_connection_string(db_username, db_password, db_host, db_port, db_name):
        connection_url = 'postgresql+psycopg2://'+db_username+':'+db_password+'@'+db_host+':'+db_port+'/'+db_name
        return connection_url

conn_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)

db_engine = create_engine(conn_url, pool_recycle=3600)

connection = db_engine.connect()

<h2>Data Collection</h2>

In [135]:
import requests
import pandas as pd
api_key = "RGAPI-ebd2eec7-be4d-409f-84d2-3fd8db8e6d81"

def get_puuid_fromSummonerId(summonerId):
    puuid = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}").json()['puuid']
    return puuid

def getLeaderBoard(league):
    get_top50_url = f"https://na1.api.riotgames.com/lol/league/v4/{league}leagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
    return requests.get(get_top50_url).json()

def get_matches(puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=20&api_key={api_key}"
    return requests.get(url).json()

def get_matchData(matchId):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"
    return requests.get(url).json()

In [11]:
chal = getLeaderBoard('challenger')
gm = getLeaderBoard('grandmaster')
master = getLeaderBoard('master')

chal_df = pd.DataFrame(chal['entries'])
gm_df = pd.DataFrame(gm['entries'])
master_df = pd.DataFrame(master['entries'])

lb_df = pd.concat([chal_df, gm_df, master_df], ignore_index=True).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
lb_df = lb_df.set_index('summonerId')

lb_df.to_sql('leaderboard', con=connection, schema='Top50SoloQ', if_exists='replace')


307

In [12]:
with db_engine.connect() as connection:
    df = pd.read_sql(text('SELECT * FROM "Top50SoloQ".leaderboard;'), connection)

df.head(300)

,summonerId,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14,2154,I,173,100,True,False,False,True
1,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8,1661,I,199,137,True,False,False,False
2,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA,1551,I,188,136,True,False,False,True
3,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U,1481,I,140,86,True,False,False,False
4,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...,1475,I,235,189,True,False,False,False
...,...,...,...,...,...,...,...,...,...
295,A9hwwGEslZDBJUbQ6DxtRuARy4agJvbhcamjSItlKKejMhk,723,I,99,49,False,False,True,False
296,ROXmKIkCunhuLDOemffR3hO-EPb5CKFK3jrUdRDr8BMHw5...,723,I,87,55,False,False,True,True
297,t4ho9gJ9qG9BUqKu7NpeCUCil4MLkkDDgzyb1VSqHonzDKM,723,I,143,120,True,False,False,False
298,UtrSNIsxG0DCLh3xJxPC8FARhI1t1JUPegs0G85I9jKeB1Y,721,I,119,74,False,False,True,True


In [13]:
import time
puuid_list = []
count = 0
for player in df.head(300)['summonerId']:
    if count == 99:
        count = 0
        time.sleep(90) #wait for ratelimit on api
    
    puuid_list.append({'puuid' : get_puuid_fromSummonerId(player),
                       'summonerId': player})
    count += 1

puuid_df = pd.DataFrame(puuid_list)

KeyError: 'puuid'

In [14]:
puuid_df.to_sql(name='summonerdata', con=connection, schema='playerId', if_exists='replace', index=True)

NameError: name 'puuid_df' is not defined

In [16]:
with db_engine.connect() as connection:
    df1 = pd.read_sql(text('SELECT * FROM "playerId".summonerdata;'), connection)
df1

,puuid,summonerId
0,Zl5uf2ERyvcZJD_4ELxzgc9SCaltAuDOjAIiuj2sdknOA1...,I_Ly9cVMnbqP3Lk_fuZbij0g57jAl1vM223hn0BY-Thlu14
1,lWMqB3Y5AuREUMTIq2uNUzYosZFcrsGm3UwKjuW8ZdQxQS...,f-f6cnj0xVapOm-KtoQZ14gpHKZBVBu5TnajCeSPNcNna2M8
2,iOI9wt8myzTiOUZpdqYumAQo1490Axk4sFvKp1ZTtz0fui...,xH7x54xRGwAXcxcwEa6mkANUFb2nYQuQcAEZuB7jOtadV1U
3,5XqBXMuSPFUrpk807hIZ5jJ-IGj2uyOgySWq6-GrXlDcmM...,ywLe0SCqv9aq3iPO_EWY9l_lnNa4_QtyMW0_w1csrlv5RhA
4,_7r_DQhl9ENq3mc9XYlpuESPeRyv3CS2lLLHrSN-WkrDvd...,cXAxF2EIp-hCfhvHDpQ6MFAb5TFHEE7aS5eFV8ZMNC_Spe...
...,...,...
85,gIkRkkSGeoDS4LK5eluixNDmvckztMfCm9AN17_IgCWGH0...,gglipYeO0beyFFmuXxDDBH5KHZE7jgSeLEnI5q4r5NqPrj...
86,IFEwVEBnESM6acu1JrkVgalzl2Bvfw-giEkbXmvwJ1IDJy...,xJEIxbDTUNxeZuj9ykXpCSAFoQcnJmwZoen5C_DjQYIvyA...
87,T10hxgwrcpB7dg9NiK0_UiNBSGa2YP9s8Xv1pYQW_hlgsq...,0K8ZCEQIPEKJkNLQMTQvkF8DrkXCuWmj_BeAp76aAIeBUd...
88,cLxnRRh2RfKTt3-XkIOHfl_ex8vNFI6dCbplwdIa3BjTaN...,aK3-PxDQ6nV8J3yj8OHF1e-lZJVCxhA_6xftXKxzCOT7lO...


In [136]:
def processMatchInfo(match): #FIND THE MATCHUP WINRATE
    redTeamChamps = []
    redTeamRoles = []
    blueTeamChamps = []
    blueTeamRoles = []
    outcome = 0
    participants = match['metadata']['participants']
    players = match['info']['participants']

    if match['info']['participants'][0]['win'] == False:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 0
        else:
            outcome = 1
    else:
        if match['info']['participants'][0]['teamId'] == 200:
            outcome = 1
        else:
            outcome = 0

    def get_index(puuid):
        index = players[participants.index(puuid)]
        return index
    
    for player in participants:
        player = get_index(player)
        if player['teamId'] == 200:
            redTeamChamps.append(player['championName'])
            redTeamRoles.append(player['teamPosition'])
        else:
            blueTeamChamps.append(player['championName'])
            blueTeamRoles.append(player['teamPosition'])
    
    match_info = pd.DataFrame([{
        'outcome' : outcome,

        #red team
        'redTeam1' : redTeamChamps[0],
        'redTeam1R' : redTeamRoles[0],

        'redTeam2' : redTeamChamps[1],
        'redTeam2R' : redTeamRoles[1],

        'redTeam3' : redTeamChamps[2],
        'redTeam3R' : redTeamRoles[2],

        'redTeam4' : redTeamChamps[3],
        'redTeam4R' : redTeamRoles[3],

        'redTeam5' : redTeamChamps[4],
        'redTeam5R' : redTeamRoles[4],

        #blue team
        'blueTeam1' : blueTeamChamps[0],
        'blueTeam1R' : blueTeamRoles[0],

        'blueTeam2' : blueTeamChamps[1],
        'blueTeam2R' : blueTeamRoles[1],

        'blueTeam3' : blueTeamChamps[2],
        'blueTeam3R' : blueTeamRoles[2],

        'blueTeam4' : blueTeamChamps[3],
        'blueTeam4R' : blueTeamRoles[3],

        'blueTeam5' : blueTeamChamps[4],
        'blueTeam5R' : blueTeamRoles[4],
    }])
    return match_info

In [137]:
uniqueMatches = []
for player in df1.head(2)['puuid']:
    for match in get_matches(player):
        if match not in uniqueMatches:
            uniqueMatches.append(match)
            
uniqueMatches

['NA1_5036518415',
 'NA1_5036511826',
 'NA1_5036496169',
 'NA1_5035936299',
 'NA1_5035925053',
 'NA1_5035906397',
 'NA1_5035470687',
 'NA1_5035428371',
 'NA1_5035402815',
 'NA1_5034824451',
 'NA1_5034811770',
 'NA1_5034780669',
 'NA1_5034768874',
 'NA1_5030494093',
 'NA1_5030442871',
 'NA1_5030412958',
 'NA1_5029179586',
 'NA1_5029163020',
 'NA1_5029143841',
 'NA1_5029123902',
 'NA1_5037307147',
 'NA1_5037284418',
 'NA1_5037217508',
 'NA1_5037203555',
 'NA1_5037155954',
 'NA1_5037145233',
 'NA1_5037132916',
 'NA1_5036652312',
 'NA1_5036042211',
 'NA1_5036019687',
 'NA1_5035968193',
 'NA1_5035164262',
 'NA1_5035124591',
 'NA1_5035098470',
 'NA1_5035071141',
 'NA1_5035035400',
 'NA1_5034998719',
 'NA1_5034957165',
 'NA1_5031000548',
 'NA1_5030997823']

In [138]:
matchall = []
all_match_df = pd.DataFrame()
for match in uniqueMatches:
    matchall.append(processMatchInfo(get_matchData(match)))

all_match_df = pd.concat(matchall)

In [139]:
all_match_df

,outcome,redTeam1,redTeam1R,redTeam2,redTeam2R,redTeam3,redTeam3R,redTeam4,redTeam4R,redTeam5,redTeam5R,blueTeam1,blueTeam1R,blueTeam2,blueTeam2R,blueTeam3,blueTeam3R,blueTeam4,blueTeam4R,blueTeam5,blueTeam5R
0,1,Kennen,TOP,Rengar,JUNGLE,Zed,MIDDLE,Kaisa,BOTTOM,Rakan,UTILITY,Udyr,TOP,Nidalee,JUNGLE,Galio,MIDDLE,Ezreal,BOTTOM,Seraphine,UTILITY
0,1,Camille,TOP,FiddleSticks,JUNGLE,Corki,MIDDLE,Kaisa,BOTTOM,Nautilus,UTILITY,Warwick,TOP,Karthus,JUNGLE,Garen,MIDDLE,Ezreal,BOTTOM,Janna,UTILITY
0,1,TwistedFate,TOP,Samira,BOTTOM,Ezreal,MIDDLE,Karthus,JUNGLE,Nautilus,UTILITY,Zeri,TOP,Khazix,JUNGLE,FiddleSticks,MIDDLE,Kaisa,BOTTOM,Alistar,UTILITY
0,0,Rumble,TOP,Viego,JUNGLE,Hwei,MIDDLE,Samira,BOTTOM,Nautilus,UTILITY,Udyr,TOP,Brand,JUNGLE,Corki,MIDDLE,Jinx,BOTTOM,Braum,UTILITY
0,1,Camille,TOP,Nunu,JUNGLE,TwistedFate,MIDDLE,Ezreal,BOTTOM,Zilean,UTILITY,Lucian,TOP,Ekko,JUNGLE,Yone,MIDDLE,MissFortune,BOTTOM,Milio,UTILITY
0,1,Camille,TOP,Ivern,JUNGLE,Lux,MIDDLE,Jhin,BOTTOM,Rakan,UTILITY,Urgot,TOP,Brand,JUNGLE,Lucian,MIDDLE,Ezreal,BOTTOM,Neeko,UTILITY
0,1,Gragas,TOP,Viego,JUNGLE,Akshan,MIDDLE,Ezreal,BOTTOM,Pyke,UTILITY,Jax,TOP,Vi,JUNGLE,Diana,MIDDLE,Jhin,BOTTOM,Bard,UTILITY
0,1,Kennen,TOP,LeeSin,JUNGLE,Galio,MIDDLE,Ezreal,BOTTOM,Bard,UTILITY,Renekton,TOP,Graves,JUNGLE,Brand,MIDDLE,MissFortune,BOTTOM,Nautilus,UTILITY
0,1,Vayne,TOP,Brand,JUNGLE,Jayce,MIDDLE,Zeri,BOTTOM,Nautilus,UTILITY,Udyr,TOP,Khazix,JUNGLE,Ezreal,MIDDLE,Kaisa,BOTTOM,Leona,UTILITY
0,1,Gangplank,TOP,Kindred,JUNGLE,Sylas,MIDDLE,Kaisa,BOTTOM,Leona,UTILITY,Jax,TOP,Khazix,JUNGLE,Ezreal,MIDDLE,Hwei,BOTTOM,Alistar,UTILITY


In [171]:
champion_data_df

,name,tier,win_rate,rank,pick_rate,ban_rate,matches
0,Aatrox,ATier,49.96%,64.41%,11.4%,16.2%,"286,783"
1,Ahri,CTier,49.66%,67.24%,8.9%,10.1%,"224,349"
2,Akali,DTier,49%,82.76%,4.8%,12.0%,"119,859"
3,Akshan,S+Tier,52.48%,5.17%,2.7%,8.6%,"67,908"
4,Alistar,STier,50.75%,27.27%,6.3%,2.3%,"158,238"
...,...,...,...,...,...,...,...
162,Zeri,CTier,49.72%,59.26%,12.0%,4.6%,"300,497"
163,Ziggs,DTier,47.88%,93.10%,1.4%,0.3%,"35,191"
164,Zilean,ATier,50.25%,40.91%,2.3%,0.7%,"57,974"
165,Zoe,ATier,50.89%,32.76%,2.5%,1.2%,"61,492"


In [174]:
test_df = champion_data_df[champion_data_df['name'].str.contains("dr", case=False)]
test_df

AttributeError: 'DataFrame' object has no attribute 'str'

In [173]:
all_match_df.reset_index(drop=True, inplace=True)
def merge_champion_stats(all_match_df, champion_data_df):
    all_matchData_df = all_match_df.copy()

    pattern = re.compile(r'^(redTeam\d+|blueTeam\d+)$')
    for index, row in all_match_df.iterrows():
        for col in all_match_df.columns:
            if pattern.match(col):
                champion_name = all_match_df.loc[index, col]
                stats = champion_data_df[champion_data_df['name'].str.lower() == champion_name.lower()]
                if not stats.empty:
                    for stat_col in champion_data_df.columns:
                        if stat_col != 'name' and stat_col != 'matches' :
                            all_matchData_df.loc[index, f"{col}_{stat_col}"] = stats[stat_col].values[0]

    for count in range(1, 6):
        all_matchData_df[f'redTeam{count}_win_rate'] = all_matchData_df[f'redTeam{count}_win_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_pick_rate'] = all_matchData_df[f'redTeam{count}_pick_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_ban_rate'] = all_matchData_df[f'redTeam{count}_ban_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'redTeam{count}_rank'] = all_matchData_df[f'redTeam{count}_rank'].str.rstrip('%').astype(float)

        all_matchData_df[f'blueTeam{count}_win_rate'] = all_matchData_df[f'blueTeam{count}_win_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_pick_rate'] = all_matchData_df[f'blueTeam{count}_pick_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_ban_rate'] = all_matchData_df[f'blueTeam{count}_ban_rate'].str.rstrip('%').astype(float)
        all_matchData_df[f'blueTeam{count}_rank'] = all_matchData_df[f'blueTeam{count}_rank'].str.rstrip('%').astype(float)

    return all_matchData_df

pd.options.display.max_columns = 100
ai_df = merge_champion_stats(all_match_df, champion_data_df)
#all_match_df.reset_index(drop=True, inplace=True)
#all_match_df.loc[0,'redTeam1']
ai_df

,outcome,redTeam1,redTeam1R,redTeam2,redTeam2R,redTeam3,redTeam3R,redTeam4,redTeam4R,redTeam5,redTeam5R,blueTeam1,blueTeam1R,blueTeam2,blueTeam2R,blueTeam3,blueTeam3R,blueTeam4,blueTeam4R,blueTeam5,blueTeam5R,redTeam1_tier,redTeam1_win_rate,redTeam1_rank,redTeam1_pick_rate,redTeam1_ban_rate,redTeam2_tier,redTeam2_win_rate,redTeam2_rank,redTeam2_pick_rate,redTeam2_ban_rate,redTeam3_tier,redTeam3_win_rate,redTeam3_rank,redTeam3_pick_rate,redTeam3_ban_rate,redTeam4_tier,redTeam4_win_rate,redTeam4_rank,redTeam4_pick_rate,redTeam4_ban_rate,redTeam5_tier,redTeam5_win_rate,redTeam5_rank,redTeam5_pick_rate,redTeam5_ban_rate,blueTeam1_tier,blueTeam1_win_rate,blueTeam1_rank,blueTeam1_pick_rate,blueTeam1_ban_rate,blueTeam2_tier,blueTeam2_win_rate,blueTeam2_rank,blueTeam2_pick_rate,blueTeam2_ban_rate,blueTeam3_tier,blueTeam3_win_rate,blueTeam3_rank,blueTeam3_pick_rate,blueTeam3_ban_rate,blueTeam4_tier,blueTeam4_win_rate,blueTeam4_rank,blueTeam4_pick_rate,blueTeam4_ban_rate,blueTeam5_tier,blueTeam5_win_rate,blueTeam5_rank,blueTeam5_pick_rate,blueTeam5_ban_rate
0,1,Kennen,TOP,Rengar,JUNGLE,Zed,MIDDLE,Kaisa,BOTTOM,Rakan,UTILITY,Udyr,TOP,Nidalee,JUNGLE,Galio,MIDDLE,Ezreal,BOTTOM,Seraphine,UTILITY,ATier,50.69,38.98,2.4,1.6,DTier,48.61,88.24,2.7,5.4,DTier,48.80,87.93,7.0,18.7,S+Tier,50.80,29.63,33.7,12.4,ATier,50.11,45.45,5.0,0.8,STier,51.88,5.88,3.4,2.3,S+Tier,50.17,50.98,9.2,12.7,S+Tier,52.42,8.62,3.8,1.5,DTier,49.56,74.07,25.2,15.1,ATier,50.63,34.09,4.7,1.1
1,1,Camille,TOP,FiddleSticks,JUNGLE,Corki,MIDDLE,Kaisa,BOTTOM,Nautilus,UTILITY,Warwick,TOP,Karthus,JUNGLE,Garen,MIDDLE,Ezreal,BOTTOM,Janna,UTILITY,S+Tier,51.58,8.47,6.3,7.2,STier,52.00,1.96,3.2,2.7,DTier,47.91,91.38,5.9,4.2,S+Tier,50.80,29.63,33.7,12.4,DTier,49.37,72.73,13.1,20.7,ATier,51.24,13.73,2.7,1.2,DTier,49.06,76.47,3.4,13.1,STier,50.45,45.76,6.8,4.7,DTier,49.56,74.07,25.2,15.1,S+Tier,51.70,9.09,5.3,3.1
2,1,TwistedFate,TOP,Samira,BOTTOM,Ezreal,MIDDLE,Karthus,JUNGLE,Nautilus,UTILITY,Zeri,TOP,Khazix,JUNGLE,FiddleSticks,MIDDLE,Kaisa,BOTTOM,Alistar,UTILITY,BTier,49.93,58.62,3.4,3.7,DTier,48.69,77.78,4.9,10.7,DTier,49.56,74.07,25.2,15.1,DTier,49.06,76.47,3.4,13.1,DTier,49.37,72.73,13.1,20.7,CTier,49.72,59.26,12.0,4.6,S+Tier,50.38,47.06,10.1,9.9,STier,52.00,1.96,3.2,2.7,S+Tier,50.80,29.63,33.7,12.4,STier,50.75,27.27,6.3,2.3
3,0,Rumble,TOP,Viego,JUNGLE,Hwei,MIDDLE,Samira,BOTTOM,Nautilus,UTILITY,Udyr,TOP,Brand,JUNGLE,Corki,MIDDLE,Jinx,BOTTOM,Braum,UTILITY,BTier,49.64,72.88,3.7,7.2,BTier,49.82,66.67,11.5,6.7,DTier,48.81,86.21,7.7,17.1,DTier,48.69,77.78,4.9,10.7,DTier,49.37,72.73,13.1,20.7,STier,51.88,5.88,3.4,2.3,DTier,47.99,88.64,2.7,15.2,DTier,47.91,91.38,5.9,4.2,S+Tier,51.36,22.22,14.9,5.9,S+Tier,51.47,13.64,6.5,3.8
4,1,Camille,TOP,Nunu,JUNGLE,TwistedFate,MIDDLE,Ezreal,BOTTOM,Zilean,UTILITY,Lucian,TOP,Ekko,JUNGLE,Yone,MIDDLE,MissFortune,BOTTOM,Milio,UTILITY,S+Tier,51.58,8.47,6.3,7.2,NaN,NaN,NaN,NaN,NaN,BTier,49.93,58.62,3.4,3.7,DTier,49.56,74.07,25.2,15.1,ATier,50.25,40.91,2.3,0.7,CTier,49.56,70.37,9.8,3.2,ATier,50.48,43.14,4.1,1.4,DTier,49.42,74.14,6.4,6.7,STier,50.50,33.33,8.9,2.6,STier,50.03,50.00,7.6,5.0
5,1,Camille,TOP,Ivern,JUNGLE,Lux,MIDDLE,Jhin,BOTTOM,Rakan,UTILITY,Urgot,TOP,Brand,JUNGLE,Lucian,MIDDLE,Ezreal,BOTTOM,Neeko,UTILITY,S+Tier,51.58,8.47,6.3,7.2,ATier,51.91,3.92,1.8,1.0,DTier,48.57,81.82,6.4,4.2,STier,50.05,44.44,15.2,3.4,ATier,50.11,45.45,5.0,0.8,ATier,51.16,13.56,2.3,0.5,DTier,47.99,88.64,2.7,15.2,CTier,49.56,70.37,9.8,3.2,DTier,49.56,74.07,25.2,15.1,BTier,49.66,65.91,1.6,0.8
6,1,Gragas,TOP,Viego,JUNGLE,Akshan,MIDDLE,Ezreal,BOTTOM,Pyke,UTILITY,Jax,TOP,Vi,JUNGLE,Diana,MIDDLE,Jhin,BOTTOM,Bard,UTILITY,BTier,49.80,69.49,3.5,2.9,BTier,49.82,66.67,11.5,6.7,S+Tier,52.48,5.17,2.7,8.6,DTier,49.56,74.07,25.2,15.1,S+Tier,50.39,38.64,7.1,20.3,S+Tier,50.10,61.02,6.4,13.1,DTier,48.40,92.16,4.1,1.3,BTier,49.92,58.82,5.3,2.6,STier,50.05,44.44,15.2,3.4,ATier,50.20,43.18,3.2,0.4
7,1,Kennen,TOP,LeeSin,JUNGLE,Galio,MIDDLE,Ezreal,BOTTOM,Bard,UTILITY,Renekton,TOP,Graves,JUNGLE,Brand,MIDDLE,

In [161]:
print(ai_df.dtypes)

outcome                  int64
redTeam1                object
redTeam1R               object
redTeam2                object
redTeam2R               object
                        ...   
blueTeam5_tier          object
blueTeam5_win_rate     float64
blueTeam5_rank         float64
blueTeam5_pick_rate    float64
blueTeam5_ban_rate     float64
Length: 71, dtype: object
